In [22]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
import missingno as msno
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#import dataset
CI_df = pd.read_excel(r"/Users/williambray/Downloads/OLD_CI Registry Dataset.copy.xlsx") 
CI_df_final = prepare_dataset(CI_df)
print(CI_df_final['CI candidate'])

0      NaN
1      1.0
2      NaN
3      NaN
4      NaN
      ... 
987    0.0
988    0.0
989    0.0
990    1.0
991    NaN
Name: CI candidate, Length: 886, dtype: float64


In [23]:
CI_df_final.to_excel('Old_CI_merge(7).xlsx', index=True)

###### #yrs no HA use could be made for some using HAcur when =yes

In [5]:
##Table of Contents
def prepare_dataset(df):
    #df=correct_laterality(df)
    ###df=create_blank_WUSM_cols(df)-- housed in select_cols
    df=create_CI_candidate_col(df)
    df=create_CNC_candidacy_col(df)
    df=create_Medicare_candidacy_col(df)
    df=select_cols(df)
    df=Fixerrors_CI_Dataset(df)
    df=CI_changes(df)
    df=year_replace(df)
    ##df=impute_length_SPHL(df)
    df=exclude_SSD(df)
    ##df=remove_empty_col(df)
    df=Laterality(df)
    df=HA_use_col(df)
    df=yrs_since_no_HA_use(df)
    df=impute_PTA(df)
    df=Age_of_onset(df)

    return df

# Select Columns

# Fix Errors and Remove Columns

In [21]:
####select columns
def select_cols(df):
    cols=[ 
        'MRN',
        #'Exclude',
        'Age',
        #'Zip9'
        #'CrowFly'
        #'ADI_State'
        #'ADI_National'
        #'Gender'
        #'Race'
        #'BMI'
        #'ACE27',
        'Etiology_ip',
        'Etiology_cl',
        'HLdur_ip',
        'HLdur_cl',
        'HLdurSP_ip',
        'HLdurSP_cl',
        'HLprg_ip',
        'HLprg_cl',
        'HAcur_ip',
        'HAcur_cl',
        'HAyrs_ip',
        'HAyrs_cl',
        'CanUseTele_ip',
        'CanUseTele_cl',
        'PriorES_ip',
        'PriorES_cl',
        'Tinnitus_ip',
        'Tinnitus_cl',
        'NoiseExp',
        'NoiseExpYrs',
        'HearingAidUse',
        'Prelingual',
        #'Tinnitus',
        'Vertigo',
        'Imbalance',
        'FamilyHistoryHL',
        'hz125_ip',
        'hz250_ip',
        'hz500_ip',
        #'hz750_ip',
        'hz1000_ip',
        #'hz1500_ip',
        'hz2000_ip',
        'hz3000_ip',
        'hz4000_ip',
        'hz6000_ip',
        'hz8000_ip',
        'hz125_cl',
        'hz250_cl',
        'hz500_cl',
        #'hz750_cl',
        'hz1000_cl',
        #'hz1500_cl',
        'hz2000_cl',
        'hz3000_cl',
        'hz4000_cl',
        'hz6000_cl',
        'hz8000_cl',
        'WRS_ip',
        'WRS_cl',
        'CNC_ip',
        'CNC_cl',
        'CNC_bi',
        'AzBioQuiet_ip',
        'AzBioQuiet_cl',
        'AzBioQuiet_bi',
        'AzBio5SNR_ip',
        'AzBio5SNR_cl',
        'AzBio5SNR_bi',
        'AzBio10SNR_ip',
        'AzBio10SNR_cl',
        'AzBio10SNR_bi',
        'MOCA',
        #'MMSE',
        'SSDCandidate',
        #'HybridCandidate'
        'CI candidate',
        'CNC candidate',
        'Medicare candidate',
        'Lat',
        
         ]
    df=df[cols]
    blank_col=[
    'L PTA (500, 1K, 2K, 4K)',
    'L PTA (500, 1K, 2K)',
    'Left Low Freq PTA (250, 500)',
    'Left Low Frequency PTA (125, 250, 500)',
    'R PTA (500, 1K, 2K, 4K)',
    'R PTA (500, 1K, 2K)',
    'Right Low Freq PTA (250, 500)',
    'Right Low Frequency PTA (125, 250, 500)',
    'Patient Data::Left Age of Onset',
    'Patient Data::Left Years since No HA Use',
    'Patient Data::Right Age of Onset',
    'Patient Data::Right Years since No HA use'
     ]
    df=df.reindex(columns=cols+blank_col)
    return df

In [19]:
##create columns
candidacy_cols=[
        'Age at CI Evaluation'
        'CNC Left',
        'CNC Right',
        'AzBio Binaural'
        'AzBio + 10dB SNR - Binaural']
#CI candidate=AZ Bio quiet or +10<60% binaural
def create_CI_candidate_col(df):
    #df['CI candidate']= df['AzBio + 10dB SNR - Binaural']<=60| (df['AzBio Binaural'] <= 60)
    df.loc[(df['AzBioQuiet_bi']<=60)|(df['AzBio10SNR_bi']<= 60),'CI candidate']=1
    df.loc[(df['AzBioQuiet_bi']>60)|(df['AzBio10SNR_bi'] > 60),'CI candidate']=0
    #df.drop(df['CI candidate']==0)
    return df
#CNC candidacy- less than 40% (worst of either ear)
def create_CNC_candidacy_col(df):
    #df['CNC candidate']=df['CNC Left']<=40|(df['CNC Right']<=40)
    df.loc[(df['CNC_ip']<=40)&(df['CNC_cl']<= 40),'CNC candidate']=1
    df.loc[(df['CNC_ip']>40)&(df['CNC_cl']>40),'CNC candidate']=0
    #df.drop(df['CNC candidate']==0)
    return df
# medicare candidate= over 65 less than 40% (az quiet or +10) binaural
def create_Medicare_candidacy_col(df):
    df['Medicare candidate'] = (((df['AzBio10SNR_bi'] <= 40) | (df['AzBioQuiet_bi'] <= 40)) & (df['Age'] >= 65))
    return df

In [8]:
#remove SSD candidates
def exclude_SSD(df):
    df['SSDCandidate'].fillna('No', inplace=True)
    df= df.loc[df['SSDCandidate'] != 'Yes']
    return df

In [9]:
def Fixerrors_CI_Dataset(df):
    nadict = {'na':np.nan,'NT':np.nan,'not tested':np.nan,'DNT':np.nan,'NA':np.nan,'dnt':np.nan, ' NA':np.nan, 'nan':np.nan, 'nA':np.nan, 'NA ':np.nan,'no info':np.nan, 'NO':'No', 'YES':'Yes','MA':np.nan,'nan':np.nan,'NA\n':np.nan}
    df = df.replace(nadict)
    return df

#CI specific changes
def CI_changes(df):
    CI_specific_changes = {'NotKnown':99,'Unknown':99,'unsure':99,'unknown, see TELEPHONE use note':99,'unknown; at least several years':99,'125':120,'CNT':0,'cnt':0,'NR':120,'NR125':120,'N':np.nan,'nr':120,'unknown':99,'unknown ':99,'unsure':99}
    df= df.replace(CI_specific_changes)
    return df
    
def year_replace(df):
#####################################replaced -0.1 with 0? ask Dr.Shew#################################################################
    years_or_monthsdict={'4 yrs':4,'10yrs':10,'3 years':3,'At Birth':0,'at least 20':20,'childhood': 10,'8 months':0.7,
                         '70s':70,'35yrs':35,'35 years':35,'60s':60,'58 years':58,'identified at 58 ':58,'possibly congenital, less than 3':0,
                         '2 months':1.6,'10-12yrs':11,'1 month':1,'3 months':0.25,'6 months':0.5,'4 months':0.3,'< 1 yr':1,'1yr':1,
                         '1-2 years':1.5,'45 years':45,'early 20s':20,'3-5 yrs':4,'Birth/congenital':0,'Birth/Congenital':0,'Birth':0,
                         'likely congenital':0,'At least 60':60,'childhood; diagnosed ~12 yrs old':12,'unknown, childhood':10,
                         'birth':0,'35 years':35,'10 years':10,'5 or 6 yrs of age':5.5,'At Birth':0,'birth, possibly; Dx at age 4':0,
                         'early 20s':20,'Likely since birth':0,'likely congenital, confirmed at 3yo':0,'childhood; diagnosed ~12 yrs old':12,
                         'unsure, possibly late 40s':47,'20s':25,'18 months':1.5,'1.5 years':1.5,'9 months':0.75, '12-15 years':13.5, 
                         'unknown, maybe 10':10,'identified at 58':58, '10 months':0.8,'72 years':72,'4 years':4,'6 months':0.5, '4 yrs':4,
                         'HL stable for about 1 year:':1,'30 years': 30,'5 yrs':5,'5 years':5,'possibly 46 years':46,'2 years':2,
                         '2 years':2,'never worn':np.nan,'unknown, maybe 10':10,'unknown, childhood':10,'1 year':1,'2-3':2.5,'Possibly in childhood':10}
    df=df.replace(years_or_monthsdict)
    return df
#childhood unknowns set to 10

In [10]:
msno.matrix(CI_df_final, labels=True)
plt.show()

NameError: name 'CI_df_final' is not defined

In [11]:
def remove_empty_col(df):
    df.dropna(how='any', inplace=True)
    return df

# Change Columns for Merging (laterality, new columns)

In [12]:
def Laterality(df):
    df['Lat'] = df['Lat'].replace({'Left':0,'Right':1})
    col_root=[
        'Etiology',
        'HLdur',
        'HLdurSP',
        'HLprg',
        'HAcur',
        'HAyrs',
        'PriorES',
        'Tinnitus',
        'WRS',
        'hz125',
        'hz250',
        'hz500',
        'hz1000',
        'hz2000',
        'hz3000',
        'hz4000',
        'hz6000',
        'hz8000',
        'CNC',
        'AzBioQuiet', 
        'AzBio10SNR',
        'AzBio5SNR',
        'CanUseTele'
    ]

    for root in col_root:
        ip_cols=root+"_ip"
        cl_cols=root+"_cl"
        left_col=root+"_L"
        right_col=root+"_R"   
        df[left_col]=df[ip_cols].where(df['Lat'] == 0, df[cl_cols])
        df[right_col]=df[ip_cols].where(df['Lat'] == 1, df[cl_cols])
    cl_ip_col=[
        'Etiology_ip',
        'Etiology_cl',
        'HLdur_ip',
        'HLdur_cl',
        'HLdurSP_ip',
        'HLdurSP_cl',
        'HLprg_ip',
        'HLprg_cl',
        'HAcur_ip',
        'HAcur_cl',
        'HAyrs_cl',
        'HAyrs_ip',
        'PriorES_ip',
        'PriorES_cl',
        'Tinnitus_ip',
        'Tinnitus_cl',
        'hz125_ip',
        'hz250_ip',
        'hz500_ip',
        'hz1000_ip',
        'hz2000_ip',
        'hz3000_ip',
        'hz4000_ip',
        'hz6000_ip',
        'hz8000_ip',
        'hz125_cl',
        'hz250_cl',
        'hz500_cl',
        'hz1000_cl',
        'hz2000_cl',
        'hz3000_cl',
        'hz4000_cl',
        'hz6000_cl',
        'hz8000_cl',
        'WRS_ip',
        'WRS_cl',
        'CNC_ip',
        'CNC_cl',
        'AzBioQuiet_ip',
        'AzBioQuiet_cl',
        'AzBio10SNR_ip',
        'AzBio10SNR_cl',
        'AzBio5SNR_cl',
        'AzBio5SNR_ip',
        'CanUseTele_ip',
        'CanUseTele_cl',
        'Lat'
        ]
    df=df.drop(cl_ip_col, axis=1)


    return df
    
        ##do same to create left cols, then drop ip and cl cols

In [13]:
def HA_use_col(df):
    df['Patient Data::Left Hearing Aid Use']=(df['HAcur_L']=='Yes')|(df['HAyrs_L']>0)
    df['Patient Data::Right Hearing Aid Use']=(df['HAcur_R']=='Yes')|(df['HAyrs_R']>0)
    df['Patient Data::Left Hearing Aid Use'].fillna(False)
    df['Patient Data::Right Hearing Aid Use'].fillna(False)
    return df 


In [14]:
###HA use bilateral vs. L and R columns

In [15]:
def yrs_since_no_HA_use(df):
    df.loc[df['HAcur_L']=='Yes','Patient Data::Left Years since No HA Use']=0
     
    df.loc[df['HAcur_R']=='Yes','Patient Data::Right Years since No HA use']=0
    return df

# Impute PTA values

In [16]:
def impute_PTA(df):
    df['L PTA (500, 1K, 2K, 4K)']=df[['hz500_L', 'hz1000_L', 'hz2000_L','hz4000_L']].mean(axis=1,skipna=True)
    df['L PTA (500, 1K, 2K)']=df[['hz500_L', 'hz1000_L', 'hz2000_L']].mean(axis=1,skipna=True)
    df['Left Low Freq PTA (250, 500)']=df[['hz250_L','hz500_L']].mean(axis=1,skipna=True)
    df['Left Low Frequency PTA (125, 250, 500)']=df[['hz500_L', 'hz250_L', 'hz125_L']].mean(axis=1,skipna=True)
    df['R PTA (500, 1K, 2K, 4K)'] =  df[['hz500_R', 'hz1000_R', 'hz2000_R','hz4000_R']].mean(axis=1,skipna=True)
    df['R PTA (500, 1K, 2K)'] =  df[['hz500_R', 'hz1000_R', 'hz2000_R']].mean(axis=1,skipna=True)
    df['Right Low Freq PTA (250, 500)'] =  df[['hz500_R', 'hz250_R']].mean(axis=1,skipna=True)
    df['Right Low Frequency PTA (125, 250, 500)']=df[['hz125_R','hz500_R', 'hz250_R']].mean(axis=1,skipna=True)
    return df
    


In [17]:
def Age_of_onset(df):
    df['Patient Data::Right Age of Onset']=df['Age']-df['HLdur_R']  
    df['Patient Data::Left Age of Onset']=df['Age']-df['HLdur_R']
    return df